In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

In [30]:
df = pd.read_csv(r'C:\Users\Dell\Downloads\bank_traffic_data.csv')

In [31]:
df.head()

,timestamp,count,window
0,2019-09-26 02:00:00+02:00,764,1
1,2019-09-27 02:00:00+02:00,0,1
2,2019-09-28 02:00:00+02:00,0,1
3,2019-09-29 02:00:00+02:00,1973,1
4,2019-09-30 02:00:00+02:00,1908,1


In [32]:
df.shape

(10013, 3)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  10013 non-null  object
 1   count      10013 non-null  int64 
 2   window     10013 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 234.8+ KB


In [34]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [35]:
df.duplicated().sum()

0

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype                                
---  ------     --------------  -----                                
 0   timestamp  10013 non-null  datetime64[ns, pytz.FixedOffset(120)]
 1   count      10013 non-null  int64                                
 2   window     10013 non-null  int64                                
dtypes: datetime64[ns, pytz.FixedOffset(120)](1), int64(2)
memory usage: 234.8 KB


In [37]:
df=df.sort_values(by=['timestamp','window']).reset_index(drop=True)

In [38]:
df.head()

,timestamp,count,window
0,2019-09-26 02:00:00+02:00,764,1
1,2019-09-26 02:00:00+02:00,320,2
2,2019-09-26 02:00:00+02:00,113,3
3,2019-09-26 02:00:00+02:00,101,4
4,2019-09-26 02:00:00+02:00,101,5


In [39]:
X_train ,y_train = df.drop('count',axis=1),df['count']

ML

In [40]:
X_train['day'] = X_train['timestamp'].dt.day
X_train['month'] = X_train['timestamp'].dt.month
X_train['year'] = X_train['timestamp'].dt.year
X_train['dayofweek'] = X_train['timestamp'].dt.dayofweek
X_train['dayofyear'] = X_train['timestamp'].dt.dayofyear
X_train['weekofyear'] = X_train['timestamp'].dt.weekofyear
X_train['is_wknd'] = X_train['timestamp'].dt.weekday // 4
X_train['is_month_start'] = X_train['timestamp'].dt.is_month_start.astype(int)
X_train['is_month_end'] = X_train['timestamp'].dt.is_month_end.astype(int)
X_train.drop('timestamp',axis=1,inplace=True)

C:\Users\Dell\Anaconda44\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [41]:
X_train.head()

,window,day,month,year,dayofweek,dayofyear,weekofyear,is_wknd,is_month_start,is_month_end
0,1,26,9,2019,3,269,39,0,0,0
1,2,26,9,2019,3,269,39,0,0,0
2,3,26,9,2019,3,269,39,0,0,0
3,4,26,9,2019,3,269,39,0,0,0
4,5,26,9,2019,3,269,39,0,0,0


In [42]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [43]:
xgb = XGBRegressor(n_estimators=8)
score = cross_val_score(xgb,X_train,y_train,scoring='neg_mean_absolute_error',cv=5,n_jobs=-1)
-score.mean()

99.47014671229458

In [44]:
lgb = LGBMRegressor(n_estimators=40)
score = cross_val_score(lgb,X_train,y_train,scoring='neg_mean_absolute_error',cv=5,n_jobs=-1)
-score.mean()

99.24757966562963

DL

In [45]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = np.array(y_train)

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [47]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [48]:
m = model_lstm.fit(X_train_series,y_train,validation_split=0.1 ,epochs=25)

Train on 9011 samples, validate on 1002 samples
Epoch 1/25
9011/9011 [==============================] - 8s 845us/sample - loss: 221.5255 - val_loss: 299.4949
Epoch 2/25
9011/9011 [==============================] - 5s 605us/sample - loss: 220.4264 - val_loss: 299.3152
Epoch 3/25
9011/9011 [==============================] - 5s 519us/sample - loss: 218.7129 - val_loss: 295.0252
Epoch 4/25
9011/9011 [==============================] - 5s 499us/sample - loss: 202.8552 - val_loss: 268.1203
Epoch 5/25
9011/9011 [==============================] - 5s 536us/sample - loss: 196.4260 - val_loss: 270.4336
Epoch 6/25
9011/9011 [==============================] - 5s 544us/sample - loss: 194.9934 - val_loss: 274.8490
Epoch 7/25
9011/9011 [==============================] - 5s 546us/sample - loss: 195.0288 - val_loss: 266.8528
Epoch 8/25
9011/9011 [==============================] - 5s 531us/sample - loss: 194.5379 - val_loss: 267.4036
Epoch 9/25
9011/9011 [==============================] - 5s 559us/sample 

**it seem that Lightgbm and Xgboost are the best**

************************************************

# Test Data

In [49]:
july_idx = df[df['timestamp'] == '2021-07-01 02:00:00+02:00'].index[0]
july = df[july_idx:].reset_index(drop=True)
augest = july.copy()
augest['timestamp'] = augest['timestamp']+pd.DateOffset(month=8)

C:\Users\Dell\Anaconda44\lib\site-packages\pandas\core\arrays\datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [50]:
augest['day'] = augest['timestamp'].dt.day
augest['month'] = augest['timestamp'].dt.month
augest['year'] = augest['timestamp'].dt.year
augest['dayofweek'] = augest['timestamp'].dt.dayofweek
augest['dayofyear'] = augest['timestamp'].dt.dayofyear
augest['weekofyear'] = augest['timestamp'].dt.weekofyear
augest['is_wknd'] = augest['timestamp'].dt.weekday // 4
augest['is_month_start'] = augest['timestamp'].dt.is_month_start.astype(int)
augest['is_month_end'] = augest['timestamp'].dt.is_month_end.astype(int)

C:\Users\Dell\Anaconda44\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [51]:
augest.head()

,timestamp,count,window,day,month,year,dayofweek,dayofyear,weekofyear,is_wknd,is_month_start,is_month_end
0,2021-08-01 02:00:00+02:00,0,1,1,8,2021,6,213,30,1,1,0
1,2021-08-01 02:00:00+02:00,0,2,1,8,2021,6,213,30,1,1,0
2,2021-08-01 02:00:00+02:00,0,3,1,8,2021,6,213,30,1,1,0
3,2021-08-01 02:00:00+02:00,0,6,1,8,2021,6,213,30,1,1,0
4,2021-08-01 02:00:00+02:00,0,7,1,8,2021,6,213,30,1,1,0


In [52]:
X = augest.drop(['timestamp','count'],axis=1)

**predict using Xgb**

In [53]:
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=8, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [54]:
pred = (xgb.predict(X)).round()

# Submission Data

In [55]:
augest['count'] = pred

In [56]:
final_prediction= augest[['timestamp','count']].to_csv('Prediction',index=False)